In [1]:
import tensorflow as tf
from tensorflow.contrib import learn 
import numpy as np

/home/youngjai/pythonVenv/tensorflow1.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/youngjai/pythonVenv/tensorflow1.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/youngjai/pythonVenv/tensorflow1.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/youngjai/pythonVenv/tenso

In [2]:

def get_word_vocab(urls, max_length_words, min_word_freq=0): 
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_length_words, min_frequency=min_word_freq) 
    x = np.array(list(vocab_processor.fit_transform(urls)))
    vocab_dict = vocab_processor.vocabulary_._mapping
    reverse_dict = dict(zip(vocab_dict.values(), vocab_dict.keys()))
    print("Size of word vocabulary: {}".format(len(reverse_dict)))
    return x, reverse_dict 

In [3]:
max_length_words = 10
min_word_freq = 0


vocab_processor = learn.preprocessing.VocabularyProcessor(max_length_words, 
                                                          min_frequency=min_word_freq)


urls = ["https://naver.com", "https://docs.ahnlab.com/ailab?df", "https://phishing", "ftp://dfsd.youngjai.juhee.com/love"]
x = np.array(list(vocab_processor.fit_transform(urls)))

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


In [4]:
x

array([[ 1,  2,  3,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  4,  5,  3,  6,  7,  0,  0,  0,  0],
       [ 1,  8,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 9, 10, 11, 12,  3, 13,  0,  0,  0,  0]])

In [5]:

#<UNK>가 나오는데, 이는,,,, 채워지는게 없을 때 그렇게 나온다. 
vocab_dict = vocab_processor.vocabulary_._mapping
print(vocab_dict)    
  

{'<UNK>': 0, 'https': 1, 'naver': 2, 'com': 3, 'docs': 4, 'ahnlab': 5, 'ailab': 6, 'df': 7, 'phishing': 8, 'ftp': 9, 'dfsd': 10, 'youngjai': 11, 'juhee': 12, 'love': 13}


In [6]:
reverse_dict = dict(zip(vocab_dict.values(), vocab_dict.keys()))
print(reverse_dict)

{0: '<UNK>', 1: 'https', 2: 'naver', 3: 'com', 4: 'docs', 5: 'ahnlab', 6: 'ailab', 7: 'df', 8: 'phishing', 9: 'ftp', 10: 'dfsd', 11: 'youngjai', 12: 'juhee', 13: 'love'}


In [7]:
processed_x = []
for i in range(x.shape[0]):
    word_url = x[i]
    raw_url = urls[i]
    print("word_url:",word_url)
    print("raw_url:", raw_url)
    words = []
    
    for w in range(len(word_url)):
        word_id = word_url[w]
        print("word_id:",word_id)
        if word_id == 0:
            words.extend(list(raw_url))
            break
        else:
            word = reverse_dict[word_id]
            print("word:",word)
            idx = raw_url.index(word)
            print("in rawURL idx:",idx)
            special_chars = list(raw_url[0:idx])
            print("special_chars", special_chars)
            words.extend(special_chars)
            words.append(word)
            print("words",words)
            
            raw_url = raw_url[idx+len(word):]
            if w == len(word_url)-1:
                words.extend(list(raw_url))
    processed_x.append(words)          
            

word_url: [1 2 3 0 0 0 0 0 0 0]
raw_url: https://naver.com
word_id: 1
word: https
in rawURL idx: 0
special_chars []
words ['https']
word_id: 2
word: naver
in rawURL idx: 3
special_chars [':', '/', '/']
words ['https', ':', '/', '/', 'naver']
word_id: 3
word: com
in rawURL idx: 1
special_chars ['.']
words ['https', ':', '/', '/', 'naver', '.', 'com']
word_id: 0
word_url: [1 4 5 3 6 7 0 0 0 0]
raw_url: https://docs.ahnlab.com/ailab?df
word_id: 1
word: https
in rawURL idx: 0
special_chars []
words ['https']
word_id: 4
word: docs
in rawURL idx: 3
special_chars [':', '/', '/']
words ['https', ':', '/', '/', 'docs']
word_id: 5
word: ahnlab
in rawURL idx: 1
special_chars ['.']
words ['https', ':', '/', '/', 'docs', '.', 'ahnlab']
word_id: 3
word: com
in rawURL idx: 1
special_chars ['.']
words ['https', ':', '/', '/', 'docs', '.', 'ahnlab', '.', 'com']
word_id: 6
word: ailab
in rawURL idx: 1
special_chars ['/']
words ['https', ':', '/', '/', 'docs', '.', 'ahnlab', '.', 'com', '/', 'ailab']
wor

In [8]:
'''
reverse_dict 는 숫자가 key이고, 해당 string이 value인 경우.
delimit mode는? 
0: delimit URL by special characters
1: delimit URL by special characters && treat each special characters as words
default로는 1이 설정되어 있다. 

get_words 라는 함수는 말그대로 URL에서 단어들(https, naver) 와 같이 특수문자로 구분되어서
얻을 수 있는 단어들, 그리고 그 특수문자들 개개를 word로 받아서 저장하는 것이다. 
'''

def get_words(x, reverse_dict, delimit_mode, urls=None): 
    
    processed_x = []
    if delimit_mode == 0: 
        for url in x: 
            # words는 걍 string 단어의 집합? 
            words = []
            for word_id in url: 
                if word_id != 0: 
                    words.append(reverse_dict[word_id])
                # 즉 해당 url이 모두 끝났다는 말임. 
                else: 
                    break
                    
            processed_x.append(words) 
    
    # 특수문자를 기준으로 나누되, 특수문자도 words로 간주한다. 
    elif delimit_mode == 1: 
        for i in range(x.shape[0]):
            word_url = x[i]     # 한 URL이 ID로 표현된것. 
            raw_url = urls[i]
            words = []
            for w in range(len(word_url)): 
                word_id = word_url[w]
                
                # 0 이라는건 padding된것. 존재하지 않으니까, string으로 된 URL을 단어로 쪼갠 (raw_url)
                # 을 그냥 바로 words에 추가시킨다. 
                if word_id == 0: 
                    words.extend(list(raw_url))
                    break
                else: 
                    # 즉 끝이 아니라, 어떤 특정단어에 mapping이 되었다.     
                    word = reverse_dict[word_id]
                    idx = raw_url.index(word) # 해당단어가 몇번째인지 알아야.. 
                    special_chars = list(raw_url[0:idx])
                    words.extend(special_chars) # 특수문자도 word로 간주하기 위해서 
                    words.append(word) # 원래 word는 그냥 넣는것이고 
                    
                    # 그 일반 단어 그 다음부터 세게끔 해서.. 
                    raw_url = raw_url[idx+len(word):]
                    if w == len(word_url) - 1: 
                        words.extend(list(raw_url))
            processed_x.append(words)
    return processed_x 

In [9]:
temp = get_words(x, reverse_dict, 1, urls)
print(temp)

[['https', ':', '/', '/', 'naver', '.', 'com'], ['https', ':', '/', '/', 'docs', '.', 'ahnlab', '.', 'com', '/', 'ailab', '?', 'df'], ['https', ':', '/', '/', 'phishing'], ['ftp', ':', '/', '/', 'dfsd', '.', 'youngjai', '.', 'juhee', '.', 'com', '/', 'love']]


In [10]:
word_reverse_dict = get_word_vocab(urls, 10,1)
sorted(list(word_reverse_dict.values()))

Size of word vocabulary: 3


AttributeError: 'tuple' object has no attribute 'values'

In [11]:
sorted(list(word_reverse_dict[1].values()))

['<UNK>', 'com', 'https']

In [12]:

'''
주어진 word에서 ngram_len 만큼씩 substring을 만들고, 한칸씩 sliding window 하는것 
파악한다. 
'''
def get_char_ngrams(ngram_len, word): 
    word = "<" + word + ">" 
    chars = list(word) 
    begin_idx = 0
    ngrams = []
    while (begin_idx + ngram_len) <= len(chars): 
        end_idx = begin_idx + ngram_len 
        ngrams.append("".join(chars[begin_idx:end_idx])) 
        begin_idx += 1 
    return ngrams 

tempList = get_char_ngrams(1,"ahnlab")
tempList

['<', 'a', 'h', 'n', 'l', 'a', 'b', '>']

In [13]:
temp

[['https', ':', '/', '/', 'naver', '.', 'com'],
 ['https',
  ':',
  '/',
  '/',
  'docs',
  '.',
  'ahnlab',
  '.',
  'com',
  '/',
  'ailab',
  '?',
  'df'],
 ['https', ':', '/', '/', 'phishing'],
 ['ftp',
  ':',
  '/',
  '/',
  'dfsd',
  '.',
  'youngjai',
  '.',
  'juhee',
  '.',
  'com',
  '/',
  'love']]

In [14]:
ngramed_x = []
worded_x = []
all_ngrams = set()
url_in_ngrams = []
url_in_words = []
word_x = temp
for url in word_x:
    words = url
    for word in words:
        print("word:",word)
        ngrams = get_char_ngrams(1, word)
        print("ng:",ngrams)
        url_in_ngrams.append(ngrams) 
        url_in_words.append(word) 
        all_ngrams.update(ngrams)
    ngramed_x.append(url_in_ngrams)
    worded_x.append(url_in_words)    

word: https
ng: ['<', 'h', 't', 't', 'p', 's', '>']
word: :
ng: ['<', ':', '>']
word: /
ng: ['<', '/', '>']
word: /
ng: ['<', '/', '>']
word: naver
ng: ['<', 'n', 'a', 'v', 'e', 'r', '>']
word: .
ng: ['<', '.', '>']
word: com
ng: ['<', 'c', 'o', 'm', '>']
word: https
ng: ['<', 'h', 't', 't', 'p', 's', '>']
word: :
ng: ['<', ':', '>']
word: /
ng: ['<', '/', '>']
word: /
ng: ['<', '/', '>']
word: docs
ng: ['<', 'd', 'o', 'c', 's', '>']
word: .
ng: ['<', '.', '>']
word: ahnlab
ng: ['<', 'a', 'h', 'n', 'l', 'a', 'b', '>']
word: .
ng: ['<', '.', '>']
word: com
ng: ['<', 'c', 'o', 'm', '>']
word: /
ng: ['<', '/', '>']
word: ailab
ng: ['<', 'a', 'i', 'l', 'a', 'b', '>']
word: ?
ng: ['<', '?', '>']
word: df
ng: ['<', 'd', 'f', '>']
word: https
ng: ['<', 'h', 't', 't', 'p', 's', '>']
word: :
ng: ['<', ':', '>']
word: /
ng: ['<', '/', '>']
word: /
ng: ['<', '/', '>']
word: phishing
ng: ['<', 'p', 'h', 'i', 's', 'h', 'i', 'n', 'g', '>']
word: ftp
ng: ['<', 'f', 't', 'p', '>']
word: :
ng: ['<', ':

In [15]:
worded_x

[['https',
  ':',
  '/',
  '/',
  'naver',
  '.',
  'com',
  'https',
  ':',
  '/',
  '/',
  'docs',
  '.',
  'ahnlab',
  '.',
  'com',
  '/',
  'ailab',
  '?',
  'df',
  'https',
  ':',
  '/',
  '/',
  'phishing',
  'ftp',
  ':',
  '/',
  '/',
  'dfsd',
  '.',
  'youngjai',
  '.',
  'juhee',
  '.',
  'com',
  '/',
  'love'],
 ['https',
  ':',
  '/',
  '/',
  'naver',
  '.',
  'com',
  'https',
  ':',
  '/',
  '/',
  'docs',
  '.',
  'ahnlab',
  '.',
  'com',
  '/',
  'ailab',
  '?',
  'df',
  'https',
  ':',
  '/',
  '/',
  'phishing',
  'ftp',
  ':',
  '/',
  '/',
  'dfsd',
  '.',
  'youngjai',
  '.',
  'juhee',
  '.',
  'com',
  '/',
  'love'],
 ['https',
  ':',
  '/',
  '/',
  'naver',
  '.',
  'com',
  'https',
  ':',
  '/',
  '/',
  'docs',
  '.',
  'ahnlab',
  '.',
  'com',
  '/',
  'ailab',
  '?',
  'df',
  'https',
  ':',
  '/',
  '/',
  'phishing',
  'ftp',
  ':',
  '/',
  '/',
  'dfsd',
  '.',
  'youngjai',
  '.',
  'juhee',
  '.',
  'com',
  '/',
  'love'],
 ['https',
  ':'

In [16]:
ngramed_x

[[['<', 'h', 't', 't', 'p', 's', '>'],
  ['<', ':', '>'],
  ['<', '/', '>'],
  ['<', '/', '>'],
  ['<', 'n', 'a', 'v', 'e', 'r', '>'],
  ['<', '.', '>'],
  ['<', 'c', 'o', 'm', '>'],
  ['<', 'h', 't', 't', 'p', 's', '>'],
  ['<', ':', '>'],
  ['<', '/', '>'],
  ['<', '/', '>'],
  ['<', 'd', 'o', 'c', 's', '>'],
  ['<', '.', '>'],
  ['<', 'a', 'h', 'n', 'l', 'a', 'b', '>'],
  ['<', '.', '>'],
  ['<', 'c', 'o', 'm', '>'],
  ['<', '/', '>'],
  ['<', 'a', 'i', 'l', 'a', 'b', '>'],
  ['<', '?', '>'],
  ['<', 'd', 'f', '>'],
  ['<', 'h', 't', 't', 'p', 's', '>'],
  ['<', ':', '>'],
  ['<', '/', '>'],
  ['<', '/', '>'],
  ['<', 'p', 'h', 'i', 's', 'h', 'i', 'n', 'g', '>'],
  ['<', 'f', 't', 'p', '>'],
  ['<', ':', '>'],
  ['<', '/', '>'],
  ['<', '/', '>'],
  ['<', 'd', 'f', 's', 'd', '>'],
  ['<', '.', '>'],
  ['<', 'y', 'o', 'u', 'n', 'g', 'j', 'a', 'i', '>'],
  ['<', '.', '>'],
  ['<', 'j', 'u', 'h', 'e', 'e', '>'],
  ['<', '.', '>'],
  ['<', 'c', 'o', 'm', '>'],
  ['<', '/', '>'],
  ['<',

In [42]:


# 즉 x가 a 안에 포함이 된다면 true 반환. 
def is_in(a,x): 
    i = bisect_left(a,x)
    if i != len(a) and a[i] == x: 
        return True 
    else: # 범위 바깥이다. 
        return False 


    
# x 배열에서 a의 위치를 찾아준다.     
def bisect_left(a, x, lo=0, hi=None):
    """Return the index where to insert item x in list a, assuming a is sorted.

    The return value i is such that all e in a[:i] have e < x, and all e in
    a[i:] have e >= x.  So if x already appears in the list, a.insert(x) will
    insert just before the leftmost x already there.

    Optional args lo (default 0) and hi (default len(a)) bound the
    slice of a to be searched.
    """

    # 즉 a가 문자열로 정렬되어 있다고 가정했을 때, 적당한 위치를 x가 들어가려고.. 들어갈 수 있는 
    # 위치를 return한다. 
    if lo < 0:
        raise ValueError('lo must be non-negative')
    if hi is None:
        hi = len(a)
    while lo < hi:
        mid = (lo+hi)//2
        if a[mid] < x: lo = mid+1
        else: hi = mid
    return lo



def get_char_ngrams(ngram_len, word): 
    word = "<" + word + ">" 
    chars = list(word) 
    begin_idx = 0
    ngrams = []
    while (begin_idx + ngram_len) <= len(chars): 
        end_idx = begin_idx + ngram_len 
        ngrams.append("".join(chars[begin_idx:end_idx])) 
        begin_idx += 1 
    return ngrams 



'''
high_freq_words = None

min_word_freq >0 이라는 것은,,,,
만약 1이라면, freq가 1이거나 그것보다 작으면 걍 <unknown>으로 다 대체하겠다는 의미임. 
단어의 출현빈도 횟수가 특정 숫자보다 낮으면 다 퉁쳐버리겠다는 단어임. 

if FLAGS["data.min_word_freq"] > 0: 
    x1, word_reverse_dict = get_word_vocab(urls, FLAGS["data.max_len_words"], FLAGS["data.min_word_freq"]) 
    high_freq_words = sorted(list(word_reverse_dict.values()))
    print("Number of words with freq >={}: {}".format(FLAGS["data.min_word_freq"], len(high_freq_words)))  

즉 min_word_freq 이상인 모든 단어들의 집합을 말한다. 

word_x : [["https", ":", "/", "naver", ".", "com"], ["https:", ... ]]
max_len_subwords : maximum number of words in a URL. 이걸 넘기면 짤리고, 못넘기면, <PADDING>으로 채워진다.
'''
def ngram_id_x(word_x, max_len_subwords, high_freq_words=None):   
    char_ngram_len = 1
    all_ngrams = set() 
    ngramed_x = []  # 각 URL -> 각 단어 -> 단어의 char 별로 seq로 묶여있다. 
    all_words = set() 
    worded_x = []
    counter = 0
    for url in word_x:
        if counter % 100000 == 0: 
            print("Processing #url {}".format(counter))
        counter += 1  
        url_in_ngrams = []
        url_in_words = []
        words = url # 각 url이 단어들의 seq로 표현되어 있으므로
        for word in words:
            # 각 단어에서 글자 하나만 가지고 있는 것.
            ngrams = get_char_ngrams(char_ngram_len, word) 
            
            # 별루 언급되지 않은 단어 => not is_in(high_freq_words, word) 로 표현한다.
            # 그 때는 <UNKNOWN>으로 대체해서 넣는다. 
            # max_len_subwords보다 길면, truncated해서 넣어야 되니까
            if (len(ngrams) > max_len_subwords) or \
                (high_freq_words is not None and len(word)>1 and not is_in(high_freq_words, word)):  
                all_ngrams.update(ngrams[:max_len_subwords])
                url_in_ngrams.append(ngrams[:max_len_subwords]) 
                all_words.add("<UNKNOWN>")
                url_in_words.append("<UNKNOWN>")
            else:     
                all_ngrams.update(ngrams)
                url_in_ngrams.append(ngrams) 
                all_words.add(word) 
                url_in_words.append(word)
        # 한 URL의 단어를 각 글자로 쪼개버린 걸 담는다.
        # 위처럼 한 걸 URL마다 담음. 
        ngramed_x.append(url_in_ngrams)
        
        ## 한 URL의 단어 그 자체를 담는다. worded_x에다가  
        worded_x.append(url_in_words) 

    ## ngramed_x : 모든 URL 목록들을 각각 쪼갠다음, 각 URL을 ngram단위로 한것들의 집합.
    ## all_ngrams 는 전체 URL 목록에 대해서 중복되지 않은 ngrams들을 뽑아낸것. 즉 https://~ 라면, 각각의 1개짜리 ngram은 h,t,p이니까.. 이것들에 id를 부여한 것이다. 
    all_ngrams = list(all_ngrams) 
    
    # 1글자씩 쪼개버린걸 dict로 바꿔서 넣는다. 
    ngrams_dict = dict()
    for i in range(len(all_ngrams)):  
        ngrams_dict[all_ngrams[i]] = i+1 # ngram id=0 is for padding ngram   
        
    # 즉 url목록에 있는 모든 글자가 이루어진 글자의 갯수     
    print("Size of ngram vocabulary: {}".format(len(ngrams_dict))) 
    
    # words를 이루는 character 한 개가 아니라, 그냥 단어별로 넣는다.
    all_words = list(all_words) 
    words_dict = dict() 
    for i in range(len(all_words)): 
        words_dict[all_words[i]] = i+1 #word id=0 for padding word 
    print("Size of word vocabulary: {}".format(len(words_dict)))
    print("Index of <UNKNOWN> word: {}".format(words_dict["<UNKNOWN>"]))        

    
    ngramed_id_x = []
    
    # ngramed_x는 [['h','t','t','p',],[':'],['/']..[]] , [['h']..[]] ]
    for ngramed_url in ngramed_x: 
        url_in_ngrams = []
        
        # 한 URL에서 (1짜리 ngram으로 단어별로 나눠진것. )
        # 따라서 ngramed_word는 URL에 속한 단어가 한 글자씩으로 쪼개진 것을 의미한다. 
        for ngramed_word in ngramed_url: 
            
            # x는 지금 한 글자임.(ngram을 1로해서...) => 그리고 그 ngram의 id를 얻으려고, dict랑 mapping한다.
            ngram_ids = [ngrams_dict[x] for x in ngramed_word] 
            
            # 그리고 그 주어진 단어(하나의 URL을 이루는 단어)를 ngram의 id로 표현했다.
            url_in_ngrams.append(ngram_ids)
        
        # 위 loop가 끝나면 한 URL을 이루는 단어가 숫자 id로 mapping되어 나타난다.
        ngramed_id_x.append(url_in_ngrams)  
        
    worded_id_x = []
    for worded_url in worded_x: 
        word_ids = [words_dict[x] for x in worded_url]
        worded_id_x.append(word_ids) 
    
    
    # ngramed_x : [ URL1[word1[h,t,t,p], word2[:], word3[/], word4[n,a,v,e,r]], URl2[]]
    # ngramed_id_x 는 위에걸 숫자 id로 바꾼 것을 말한다. 
    # worded_id_x 는 한 URL을 특수문자(ex: / , :) 하나를 하나의 word로 포함시켜서, 이 word 목록에다가 겹치지 않게 해서 id를 mapping시킨걸 의미 
    # words_dict 는 이걸 mapping 할 수 있게 해주는 dictionary이다. 
    return ngramed_id_x, ngrams_dict, worded_id_x, words_dict 

# data.max_len_words 는 단어의 최디 길이가 아니라, URL을 단어 단위로 표현할 때, 최대 몇개까지의 단어로 이루어질 수 있는가를 표현.
min_freq = 1
if min_freq > 0: 
    x1, word_reverse_dict = get_word_vocab(urls, 20, min_freq) 
    high_freq_words = sorted(list(word_reverse_dict.values()))
    print("Number of words with freq >={}: {}".format(min_freq, len(high_freq_words)))  

x, word_reverse_dict = get_word_vocab(urls, 200) 
word_x = get_words(x, word_reverse_dict, 1, urls)    
    
ngramed_id_x, ngrams_dict, worded_id_x, words_dict = ngram_id_x(word_x, 20, high_freq_words)


Size of word vocabulary: 3
Number of words with freq >=1: 3
Size of word vocabulary: 14
Processing #url 0
Size of ngram vocabulary: 27
Size of word vocabulary: 7
Index of <UNKNOWN> word: 3


In [18]:
ngramed_id_x

[[[19, 18, 14, 14, 7, 3, 8],
  [19, 22, 8],
  [19, 2, 8],
  [19, 2, 8],
  [19, 25, 21, 17, 10, 24, 8],
  [19, 12, 8],
  [19, 26, 1, 5, 8]],
 [[19, 18, 14, 14, 7, 3, 8],
  [19, 22, 8],
  [19, 2, 8],
  [19, 2, 8],
  [19, 13, 1, 26, 3, 8],
  [19, 12, 8],
  [19, 21, 18, 25, 9, 21, 4, 8],
  [19, 12, 8],
  [19, 26, 1, 5, 8],
  [19, 2, 8],
  [19, 21, 16, 9, 21, 4, 8],
  [19, 23, 8],
  [19, 13, 15, 8]],
 [[19, 18, 14, 14, 7, 3, 8],
  [19, 22, 8],
  [19, 2, 8],
  [19, 2, 8],
  [19, 7, 18, 16, 3, 18, 16, 25, 27, 8]],
 [[19, 15, 14, 7, 8],
  [19, 22, 8],
  [19, 2, 8],
  [19, 2, 8],
  [19, 13, 15, 3, 13, 8],
  [19, 12, 8],
  [19, 11, 1, 6, 25, 27, 20, 21, 16, 8],
  [19, 12, 8],
  [19, 20, 6, 18, 10, 10, 8],
  [19, 12, 8],
  [19, 26, 1, 5, 8],
  [19, 2, 8],
  [19, 9, 1, 17, 10, 8]]]

In [19]:
ngrams_dict

{'o': 1,
 '/': 2,
 's': 3,
 'b': 4,
 'm': 5,
 'u': 6,
 'p': 7,
 '>': 8,
 'l': 9,
 'e': 10,
 'y': 11,
 '.': 12,
 'd': 13,
 't': 14,
 'f': 15,
 'i': 16,
 'v': 17,
 'h': 18,
 '<': 19,
 'j': 20,
 'a': 21,
 ':': 22,
 '?': 23,
 'r': 24,
 'n': 25,
 'c': 26,
 'g': 27}

In [37]:
worded_id_x

[[1, 6, 2, 2, 3, 4, 5],
 [1, 6, 2, 2, 3, 4, 3, 4, 5, 2, 3, 7, 3],
 [1, 6, 2, 2, 3]]

In [38]:
words_dict

{'https': 1, '/': 2, '<UNKNOWN>': 3, '.': 4, 'com': 5, ':': 6, '?': 7}

In [20]:

# 걍 URL을 한 글자씩 잘라가지고, char_dict(한 글자당 mapping되어 있는 숫자 id) 에서 찾아가지고 
# character 대신에 그냥 숫자 id로 표현하는 것임.
def char_id_x(urls, char_dict, max_len_chars): 
    chared_id_x = []
    for url in urls: 
        url = list(url) 
        url_in_char_id = []
        l = min(len(url), max_len_chars)
        
        # 길어봤자 200개까지만 truncated할거라서, 거기까지만 idx를 
        for i in range(l): 
            c = url[i] 
            
            # 걍 그 글자(character) 에 해당하는 숫자 id를 알고 싶은 것임. 
            try:
                c_id = char_dict[c] 
            except KeyError:
                c_id = 0
            url_in_char_id.append(c_id) 
        chared_id_x.append(url_in_char_id) 
    return chared_id_x 

chars_dict = ngrams_dict


# max_len_chars : 한 URL이 character를 가질 수 있는 최대 갯수 : 200
chared_id_x = char_id_x(urls, chars_dict, 200)

In [21]:
chared_id_x

[[18, 14, 14, 7, 3, 22, 2, 2, 25, 21, 17, 10, 24, 12, 26, 1, 5],
 [18,
  14,
  14,
  7,
  3,
  22,
  2,
  2,
  13,
  1,
  26,
  3,
  12,
  21,
  18,
  25,
  9,
  21,
  4,
  12,
  26,
  1,
  5,
  2,
  21,
  16,
  9,
  21,
  4,
  23,
  13,
  15],
 [18, 14, 14, 7, 3, 22, 2, 2, 7, 18, 16, 3, 18, 16, 25, 27],
 [15,
  14,
  7,
  22,
  2,
  2,
  13,
  15,
  3,
  13,
  12,
  11,
  1,
  6,
  25,
  27,
  20,
  21,
  16,
  12,
  20,
  6,
  18,
  10,
  10,
  12,
  26,
  1,
  5,
  2,
  9,
  1,
  17,
  10]]

In [35]:
from tflearn.data_utils import to_categorical 
'''
pos_x 는 URL이 +인 idx 번호가 들어있는 배열이다. 
neg_x 는 URL이 -인 idx 번호가 들어있는 배열이다. 
'''


labels = [-1,-1,1,1,1,-1,1,-1,-1,1,1,1,1,-1,-1,-1,1,-1]
pos_x = []
neg_x = []
for i in range(len(labels)):
    label = labels[i] 
    if label == 1: 
        pos_x.append(i)
    else: 
        neg_x.append(i)
print("Overall Mal/Ben split: {}/{}".format(len(pos_x), len(neg_x)))
pos_x = np.array(pos_x) 
neg_x = np.array(neg_x) 


def prep_train_test(pos_x, neg_x, dev_pct): 
    np.random.seed(10) 
    
    # 말그대로 주어진 +인 idx목록에서 순서를 뒤죽박죽 섞는다.(permutation역할) 
    shuffle_indices=np.random.permutation(np.arange(len(pos_x)))
    
    # idx가 섞였다. (1,2,3,4 이런식의 indices가 나오지 않는다.)
    pos_x_shuffled = pos_x[shuffle_indices]
    dev_idx = -1 * int(dev_pct * float(len(pos_x)))
    
    # 어차피 지금 순서가 뒤죽박죽으로 섞여있는 상태니깐, 처음부터 그 비율에 맞는 수까지만 고른다. 
    pos_train = pos_x_shuffled[:dev_idx]
    pos_test = pos_x_shuffled[dev_idx:]

    np.random.seed(10)
    shuffle_indices=np.random.permutation(np.arange(len(neg_x)))
    neg_x_shuffled = neg_x[shuffle_indices]
    dev_idx = -1 * int(dev_pct * float(len(neg_x)))
    neg_train = neg_x_shuffled[:dev_idx]
    neg_test = neg_x_shuffled[dev_idx:] 

    # 그냥 idx목록들이 담겨져있다? 
    x_train = np.array(list(pos_train) + list(neg_train))
    y_train = len(pos_train)*[1] + len(neg_train)*[0]
    x_test = np.array(list(pos_test) + list(neg_test))
    y_test = len(pos_test)*[1] + len(neg_test)*[0]

    # y를 categorical 하게 해주는게 무슨 의미?
    y_train = to_categorical(y_train, nb_classes=2)
    y_test = to_categorical(y_test, nb_classes=2) 

    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(x_train)))
    x_train = x_train[shuffle_indices]
    y_train = y_train[shuffle_indices]

    np.random.seed(10) 
    shuffle_indices = np.random.permutation(np.arange(len(x_test)))
    x_test = x_test[shuffle_indices]
    y_test = y_test[shuffle_indices] 
    
    print("Train Mal/Ben split: {}/{}".format(len(pos_train), len(neg_train)))
    print("Test Mal/Ben split: {}/{}".format(len(pos_test), len(neg_test)))
    print("Train/Test split: {}/{}".format(len(y_train), len(y_test)))
    print("Train/Test split: {}/{}".format(len(x_train), len(x_test)))

    # shuffled_indices로 한번에 하는거라 그 순서는 잘 mapping이 된다.
    # x쪽 데이터는 label의 idx이고, test 해당 label의 결과를 one-hot encoding한 것이다. 
    return x_train, y_train, x_test, y_test 

# x_train,
x_train, y_train, x_test, y_test = prep_train_test(pos_x, neg_x, 0.2)

Overall Mal/Ben split: 9/9
Train Mal/Ben split: 8/8
Test Mal/Ben split: 1/1
Train/Test split: 16/2
Train/Test split: 16/2


In [38]:
pos_x_shuffled = pos_x[shuffle_indices]
pos_train = pos_x_shuffled[:dev_idx]
shuffle_indices=np.random.permutation(np.arange(len(neg_x)))
neg_x_shuffled = neg_x[shuffle_indices]
dev_idx = -1 * int(0.2 * float(len(neg_x)))
neg_train = neg_x_shuffled[:dev_idx]
y_train = len(pos_train)*[1] + len(neg_train)*[0]
y_train
# y_train = to_categorical(y_train, nb_classes=2)

[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [39]:
y_train = to_categorical(y_train, nb_classes=2)
y_train

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [40]:
x_train = np.array(list(pos_train) + list(neg_train))
x_train

array([ 2,  4, 17,  5, 13, 14,  7,  1,  0, 15])

In [36]:
dev_idx = -1 * int(0.2 * float(len(neg_x)))
dev_idx

-1

In [37]:
int(0.2 * float(len(neg_x)))

1

In [41]:

# 걍 train에 있는 idx에 맞게 ngram 된것 mapping 시켜서 맞춰주는것 
def get_ngramed_id_x(x_idxs, ngramed_id_x): 
    output_ngramed_id_x = [] 
    for idx in x_idxs:  
        output_ngramed_id_x.append(ngramed_id_x[idx])
    return output_ngramed_id_x

x_train_char = get_ngramed_id_x(x_train, ngramed_id_x) 

IndexError: list index out of range

## yield ? 

batch 로 만들때 자주 쓰인다. 
엄청나게 방대한 데이터를 batch로 쪼개면 그게 다 memory에 올라가 있게 된다.      
그런데, 이렇게 yield로 하면 필요할 때마다 수행을 하게 된다. batch 별로 순환하게 될텐데, 모두 순환해서 batch들의 묵음을 전달하는 것이 아니라, 첫 batch 하고 나면, yield 되서 해당 batch에 대한 연산을 한다.     
그리고 나서, 다시 yield가 있는 함수로 넘어가서 두번째 batch를 계산하게 된다.      

yield가 있는 함수를 처음부터 싹다 계산해서 결과값을 얻어내는 것이 아니라, 그때그때마다 계산해서 값을 얻어내는 방식이다. 

어떤 iterable을 만드는데, iterable 내의 각 원소를 접근할 때는 그냥 순차적으로 간다는 말이다. 
한 번에 다 memory에 올리는 것이 아니라..      



In [43]:
def number_generator(n):
    print("Function Start")
    while n <6:
        yield n
        n+= 1
    print("Function End")
     
if __name__== "__main__":
    for i in number_generator(0):
        print(i)



Function Start
0
1
2
3
4
5
Function End


In [46]:
def number_generator(n):
    print("Function Start")
    while n <6:
        n+= 1
        return n
    print("Function End")
     
if __name__== "__main__":
    for idx in number_generator(0):
        print(idx)

Function Start


TypeError: 'int' object is not iterable